# Data preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

## Obtaining base train and test dataframes

### Retrieving both train and test application CSV files from Kaggle API

In [2]:
%run ../data/1.0-cbw-data-retrieval.ipynb

### Creating train and test dataframes

In [3]:
def get_dataframes():
  """Return a tuple containing train and test dataframes."""
  train = pd.read_csv('../../data/external/application_train.csv')
  test = pd.read_csv('../../data/external/application_test.csv')
  return train, test

In [4]:
(train, test) = get_dataframes()

In [5]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Moving target to last column in train dataset

In [7]:
def position_target_column(train):
  """Return train dataframe with target as last column.

  Keyword arguments:
  train -- the train dataframe
  """
  target_col = train.pop('TARGET')
  train['TARGET'] = target_col
  return train

In [8]:
train = position_target_column(train)

### Dropping unused ID column

In [9]:
def drop_id_column(train, test):
  """Return a tuple containing train and test dataframes without id column.

  Keyword arguments:
  train -- the train dataframe
  test -- the test dataframe
  """
  train = train.drop(['SK_ID_CURR'], axis=1)
  test = test.drop(['SK_ID_CURR'], axis=1)
  return train, test

In [10]:
(train, test) = drop_id_column(train, test)

### Organizing test set columns based on train set column order

In [11]:
def reorder_test_columns(train, test):
  """Return test dataframe with columns organized following train dataframe columns order.

  Keyword arguments:
  train -- the train dataframe
  test -- the test dataframe
  """
  test = test[train.drop(['TARGET'], axis=1).columns]
  return test

In [12]:
test = reorder_test_columns(train, test)

## Taking care of missing data

In [13]:
def impute_train_missing_data(train):
  """
  Return tuple containing train dataframe with median imputed in place of missing numerical values 
  and a Series with its numerical columns.

  Keyword arguments:
  train -- the train dataframe
  """
  imputer = SimpleImputer(missing_values=np.nan, strategy='median')
  x_dtypes_train = train.dtypes[:-1]
  num_cols_train = x_dtypes_train == np.number
  X_train = train.iloc[:, :-1].values
  imputer.fit(X_train[:, num_cols_train])
  X_train[:, num_cols_train] = imputer.transform(X_train[:, num_cols_train])
  train.iloc[:, :-1] = X_train
  return train, num_cols_train

In [14]:
(train, num_cols_train) = impute_train_missing_data(train)

In [15]:
def impute_test_missing_data(test):
  """
  Return tuple containing test dataframe with median imputed in place of missing numerical values 
  and a Series with its numerical columns.

    Keyword arguments:
    test -- the test dataframe
    """
  imputer = SimpleImputer(missing_values=np.nan, strategy='median')
  x_dtypes_test = test.dtypes
  num_cols_test = x_dtypes_test == np.number
  X_test = test.iloc[:, :].values
  imputer.fit(X_test[:, num_cols_test])
  X_test[:, num_cols_test] = imputer.transform(X_test[:, num_cols_test])
  test.iloc[:, :] = X_test
  return test, num_cols_test

In [16]:
(test, num_cols_test) = impute_test_missing_data(test)

### Getting text features Na rows percentage

In [17]:
def get_train_na_percentages(train):
  """
  Return a Series with the percentage of Na values per columns in train dataframe.
  Must be called just after impute_train_missing_data().

  Keyword arguments:
  train -- the train dataframe
  """
  na_cols_pctg_train = train[train.columns[train.isna().sum() > 0]].isna().sum() / train.shape[0]
  return na_cols_pctg_train

In [18]:
na_cols_pctg_train = get_train_na_percentages(train)
na_cols_pctg_train

NAME_TYPE_SUITE        0.004201
OCCUPATION_TYPE        0.313455
FONDKAPREMONT_MODE     0.683862
HOUSETYPE_MODE         0.501761
WALLSMATERIAL_MODE     0.508408
EMERGENCYSTATE_MODE    0.473983
dtype: float64

### Dropping text features Na rows

In [19]:
def drop_textual_feat_na_rows(train, test):
  """Return a tuple containing train and test dataframes without textual features Na rows.

  Keyword arguments:
  train -- the train dataframe
  test -- the test dataframe
  """
  train = train.dropna(axis=0)
  test = test.dropna(axis=0)
  return train, test

In [20]:
(train, test) = drop_textual_feat_na_rows(train, test)

## Feature scaling

In [21]:
def standardize_train(train, num_cols_train):
  """Return the train dataframe with standardized numerical features (not the encoded textual dimensions).

  Keyword arguments:
  train -- the train dataframe
  """
  sc = StandardScaler()
  X_train = train.iloc[:, :-1].values
  X_train[:, num_cols_train] = sc.fit_transform(X_train[:, num_cols_train])
  train.iloc[:, :-1] = X_train
  return train

In [22]:
train = standardize_train(train, num_cols_train)

In [23]:
def standardize_test(test, num_cols_test):
  """Return the test dataframe with standardized numerical features (not the encoded textual dimensions).

  Keyword arguments:
  test -- the test dataframe
  """
  sc = StandardScaler() # standardization implies values between approximately -3 and 3
  X_test = test.iloc[:, :].values
  X_test[:, num_cols_test] = sc.fit_transform(X_test[:, num_cols_test]) # we don't standardize encoded textual dimensions.
  test.iloc[:, :] = X_test
  return test

In [24]:
test = standardize_test(test, num_cols_test)

## Encoding categorical data

In [25]:
train.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,Cash loans,M,N,Y,0,0.112486,-0.545235,-0.275900,-0.569221,Unaccompanied,...,0,0,0,-0.073559,-0.059606,-0.160059,-0.290937,-0.396781,-0.425172,1
1,Cash loans,F,N,N,0,0.704502,1.526865,0.434724,1.412794,Family,...,0,0,0,-0.073559,-0.059606,-0.160059,-0.290937,-0.396781,-1.004679,0
13,Cash loans,M,Y,N,1,0.309825,0.650662,-0.000257,0.312947,Unaccompanied,...,0,0,0,-0.073559,-0.059606,-0.160059,-0.290937,-0.396781,-0.425172,0
14,Cash loans,F,N,Y,0,-0.005917,0.312391,0.246019,0.267120,Unaccompanied,...,0,0,0,-0.073559,-0.059606,-0.160059,-0.290937,-0.396781,-0.425172,0
18,Revolving loans,F,N,Y,0,-0.676868,-1.127208,-1.363063,-1.061861,Other_A,...,0,0,0,-0.073559,-0.059606,-0.160059,-0.290937,-0.396781,-1.004679,0


In [26]:
test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
3,Cash loans,F,N,Y,2,1.059309,2.562260,1.028660,2.935280,Unaccompanied,...,0,0,0,0,-0.040332,-0.032825,-0.053987,-0.077883,-0.729849,0.649564
5,Cash loans,F,Y,Y,0,0.647693,1.007165,0.177099,0.841447,Unaccompanied,...,0,0,0,0,-0.040332,-0.032825,-0.053987,-0.077883,0.749922,0.052319
15,Cash loans,M,Y,Y,0,-0.175538,-0.669483,-0.585425,-0.759719,Unaccompanied,...,0,0,0,0,-0.040332,-0.032825,-0.053987,-0.077883,0.749922,0.649564
18,Cash loans,F,Y,Y,1,0.236077,-0.328294,-0.476987,-0.390220,Unaccompanied,...,0,0,0,0,-0.040332,-0.032825,-0.053987,-0.077883,3.709463,1.246809
20,Cash loans,F,N,Y,0,-0.916446,-0.849631,-0.730010,-0.759719,Unaccompanied,...,0,0,0,0,-0.040332,-0.032825,-0.053987,-0.077883,0.749922,0.052319


### Encoding the independent variables

In [27]:
def get_textual_column_indexes(train, test):
  """Return a tuple containing an ndarray with train and test textual column indexes.

  Keyword arguments:
  train -- the train dataframe
  test -- the test dataframe
  """
  txt_cols_train = train.select_dtypes('object').columns
  txt_indexes_train = train.columns.get_indexer(txt_cols_train)
  txt_cols_test = test.select_dtypes('object').columns
  txt_indexes_test = test.columns.get_indexer(txt_cols_test)
  return txt_indexes_train, txt_indexes_test

In [28]:
(txt_indexes_train, txt_indexes_test) = get_textual_column_indexes(train, test)

In [29]:
txt_indexes_train

array([ 0,  1,  2,  3,  9, 10, 11, 12, 13, 26, 30, 38, 84, 85, 87, 88],
      dtype=int64)

In [30]:
txt_indexes_test

array([ 0,  1,  2,  3,  9, 10, 11, 12, 13, 26, 30, 38, 84, 85, 87, 88],
      dtype=int64)

In [31]:
def one_hot_encode_train(train, txt_indexes_train):
  """Return the train dataframe with one-hot-encoded textual features.

  Keyword arguments:
  train -- the train dataframe
  txt_indexes_train -- ndarray of train textual column indexes
  """
  train_dummies = pd.get_dummies(train.iloc[:, txt_indexes_train])
  train.drop(train.select_dtypes('object').columns, axis=1, inplace=True)
  train = pd.concat([train, train_dummies], axis=1)
  train = position_target_column(train)
  return train

In [32]:
def one_hot_encode_test(test, txt_indexes_test):
  """Return the test dataframe with label-encoded textual features.

  Keyword arguments:
  test -- the test dataframe
  txt_indexes_test -- ndarray of test textual column indexes
  """
  test_dummies = pd.get_dummies(test.iloc[:, txt_indexes_test])
  test.drop(test.select_dtypes('object').columns, axis=1, inplace=True)
  test = pd.concat([test, test_dummies], axis=1)
  return test

In [33]:
train = one_hot_encode_train(train, txt_indexes_train)

In [34]:
test = one_hot_encode_test(test, txt_indexes_test)

In [35]:
# Align the training and testing data, keep only columns present in both dataframes
target_col = train['TARGET']
train, test = train.align(test, join = 'inner', axis = 1)
train = pd.concat([train, target_col], axis=1)

In [36]:
train.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
0,0,0.112486,-0.545235,-0.275900,-0.569221,-0.253586,-9461,-637,0.359172,-2120,...,0,0,0,0,0,1,0,1,0,1
1,0,0.704502,1.526865,0.434724,1.412794,-1.210008,-16765,-1188,1.090039,-291,...,1,0,0,0,0,0,0,1,0,0
13,1,0.309825,0.650662,-0.000257,0.312947,-0.390782,-14086,-3028,1.251233,-4911,...,0,0,0,0,1,0,0,1,0,0
14,0,-0.005917,0.312391,0.246019,0.267120,-0.804813,-14583,-203,1.259545,-2056,...,0,0,0,0,1,0,0,1,0,0
18,0,-0.676868,-1.127208,-1.363063,-1.061861,1.464901,-17718,-7804,-1.155698,-1259,...,0,0,0,0,0,1,0,1,0,0


In [37]:
test.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
3,2,1.059309,2.562260,1.028660,2.935280,0.196341,-13976,-1866,0.830175,-4208,...,0,0,0,0,0,1,0,0,1,0
5,0,0.647693,1.007165,0.177099,0.841447,0.120865,-18604,-12009,-0.379934,-2027,...,0,1,0,0,0,0,0,0,1,0
15,0,-0.175538,-0.669483,-0.585425,-0.759719,0.575500,-13563,-1007,-0.263216,-4044,...,0,0,0,0,0,0,1,0,1,0
18,1,0.236077,-0.328294,-0.476987,-0.390220,0.120865,-10962,-1883,1.389072,-1721,...,0,0,0,0,0,1,0,0,1,0
20,0,-0.916446,-0.849631,-0.730010,-0.759719,-0.978692,-10507,-2780,-0.072997,-2729,...,0,0,0,0,0,1,0,0,1,0


## Exporting preprocessed data to CSV files

In [38]:
def export_dataframes_to_csv_files(train, test):
  """Export train and test dataframes to CSV files to ./data/processed path.

  Keyword arguments:
  train -- the train dataframe
  test -- the test dataframe
  """
  train.to_csv('../../data/processed/processed_application_train.csv', index=False)
  test.to_csv('../../data/processed/processed_application_test.csv', index=False)

In [39]:
export_dataframes_to_csv_files(train, test)